# QA System Testing

This notebook is dedicated to testing the QA system built on AWS. It will cover various aspects of the system, including:
- Testing the document retrieval process
- Validating the accuracy of answers generated by the LLM
- Measuring the latency of the system under different loads

## Setup
Before running the tests, ensure that the AWS services are properly configured and the necessary documents are uploaded to S3.

In [ ]:
# # Import necessary libraries
# import boto3
# import time
# import json

# # Initialize AWS clients
# s3_client = boto3.client('s3')
# lambda_client = boto3.client('lambda')

# # Define constants
# BUCKET_NAME = 'genaiprojectawsbucket'
# LAMBDA_FUNCTION_NAME = 'your-lambda-function-name'

# # Function to test document retrieval
# def test_document_retrieval(query):
#     response = lambda_client.invoke(
#         FunctionName=LAMBDA_FUNCTION_NAME,
#         Payload=json.dumps({'query': query})
#     )
#     return json.loads(response['Payload'].read())

# # Function to measure latency
# def measure_latency(query):
#     start_time = time.time()
#     answer = test_document_retrieval(query)
#     latency = time.time() - start_time
#     return answer, latency

# # Example queries for testing
# queries = [
#     'What are the compliance requirements for data protection?',
#     'Summarize the key points of the contract.',
#     'What is the deadline for submitting the report?'
# ]

# # Run tests
# for query in queries:
#     answer, latency = measure_latency(query)
#     print(f'Query: {query}')
#     print(f'Answer: {answer}')
#     print(f'Latency: {latency:.2f} seconds')
#     print('-' * 50)

In [1]:
# Import necessary libraries
import boto3
import time
import json
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# Initialize AWS clients
s3_client = boto3.client('s3')

# Define constants
BUCKET_NAME = 'genaiprojectawsbucket'  # Update with your bucket name
DOCUMENT_NAME = 'doc1.pdf'  # Update with your document name

# Load embeddings from previous notebook
embeddings_df = pd.read_csv('embeddings.csv')
embeddings = embeddings_df.values.astype('float32')

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# Load text chunks corresponding to embeddings
def load_extracted_text():
    try:
        # Try to load from S3
        response = s3_client.get_object(
            Bucket=BUCKET_NAME, Key='doc1_text.txt')
        text = response['Body'].read().decode('utf-8')
        text_chunks = text.split('\n')
    except Exception as e:
        # If that fails, try loading from local file
        try:
            with open('extracted_text.txt', 'r') as f:
                text = f.read()
                text_chunks = text.split('\n')
        except:
            # Use a placeholder if no file exists
            text_chunks = ["No text available"]

    return text_chunks


text_chunks = load_extracted_text()
print(f"Loaded {len(text_chunks)} text chunks")

Loaded 95 text chunks


In [3]:
# Function to retrieve relevant text chunks
def retrieve_documents(query, k=3):
    # Create query embedding
    query_embedding = model.encode([query])[0].astype('float32')
    query_embedding = query_embedding.reshape(1, -1)

    # Search FAISS index
    distances, indices = index.search(query_embedding, k)

    # Get corresponding text chunks
    retrieved_chunks = [text_chunks[i]
                        for i in indices[0] if i < len(text_chunks)]

    return retrieved_chunks, distances[0]

# Function for simple QA


def ask_question(query):
    retrieved_chunks, scores = retrieve_documents(query)

    # Build context from retrieved chunks
    context = "\n".join(retrieved_chunks)

    # In a full implementation, you would send this to an LLM
    # For this simple test, we'll just return the retrieved chunks
    return {
        "query": query,
        "context": context,
        "answer": f"Based on the retrieved information, I found these relevant passages:\n\n{context}"
    }

# Function to measure latency


def measure_latency(query):
    start_time = time.time()
    answer = ask_question(query)
    latency = time.time() - start_time
    return answer, latency

In [4]:
# Example queries for testing
queries = [
    'What is the main topic of this document?',
    'Summarize the key points.',
    'What information does this document contain?'
    # Add your own relevant questions here
]

# Run tests
for query in queries:
    answer, latency = measure_latency(query)
    print(f'Query: {query}')
    print(f'Answer: {answer["answer"][:500]}...')  # Show first 500 chars
    print(f'Latency: {latency:.2f} seconds')
    print('-' * 50)

Query: What is the main topic of this document?
Answer: Based on the retrieved information, I found these relevant passages:

3.5 For the term of the agreement, upon presentation of expense statements
1. Employment, Duties and Acceptance.
consideration, the receipt and sufficiency of which are hereby expressly...
Latency: 2.08 seconds
--------------------------------------------------
Query: Summarize the key points.
Answer: Based on the retrieved information, I found these relevant passages:

working interest in each horizontal well), and
1. Employment, Duties and Acceptance.
services described above....
Latency: 0.02 seconds
--------------------------------------------------
Query: What information does this document contain?
Answer: Based on the retrieved information, I found these relevant passages:

3.5 For the term of the agreement, upon presentation of expense statements
1. Employment, Duties and Acceptance.
1.3 Any transactions or agreements between the Company and Petroleum...